In [1]:
import numpy as  np
import sys
import pprint

# Case for independent measurements

In [2]:
np.random.seed(0)    
mean = [0, 0, 0]
cov = [[1, 0, 0], 
       [0, 1, 0],
       [0, 0, 1]
       ]  # diagonal covariance
num_samples = 10 
# Generate 1000 samples
samples = np.random.multivariate_normal(mean, 
                                        cov, 
                                        num_samples)

mean_est = np.mean(samples, axis=0)
cov_est = np.cov(samples, rowvar=False)

pprint.pp(samples)

array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788],
       [ 0.95008842, -0.15135721, -0.10321885],
       [ 0.4105985 ,  0.14404357,  1.45427351],
       [ 0.76103773,  0.12167502,  0.44386323],
       [ 0.33367433,  1.49407907, -0.20515826],
       [ 0.3130677 , -0.85409574, -2.55298982],
       [ 0.6536186 ,  0.8644362 , -0.74216502],
       [ 2.26975462, -1.45436567,  0.04575852],
       [-0.18718385,  1.53277921,  1.46935877]])


In [3]:
cov_diff = cov - cov_est
print("Mean estimate: ", mean_est)
print("Covariance estimate: ", cov_est)
print("Covariance difference: ", cov_diff)


Mean estimate:  [ 0.95096016  0.39649097 -0.01888178]
Covariance estimate:  [[ 0.72933993 -0.18482682 -0.10788968]
 [-0.18482682  1.14532262  0.25984144]
 [-0.10788968  0.25984144  1.50041999]]
Covariance difference:  [[ 0.27066007  0.18482682  0.10788968]
 [ 0.18482682 -0.14532262 -0.25984144]
 [ 0.10788968 -0.25984144 -0.50041999]]


In [4]:
# now we remove columns
selected_samples = samples[:, :2]

# re estimate the mean and covariance
mean_est_selected = np.mean(selected_samples, axis=0)
cov_est_selected = np.cov(selected_samples, rowvar=False)

print("Mean estimate: ", mean_est_selected)
print("Covariance estimate: ", cov_est_selected)

Mean estimate:  [0.95096016 0.39649097]
Covariance estimate:  [[ 0.72933993 -0.18482682]
 [-0.18482682  1.14532262]]


## If the measurements are correlated

In [5]:
np.random.seed(1)  
mean = [0, 0, 0]
cov = [[1, 0.1, 0.1], 
       [0.1, 1, 0.1],
       [0.1, 0.1, 1]
       ]  # diagonal covariance
num_samples = 1000 
# Generate 1000 samples
samples = np.random.multivariate_normal(mean, 
                                        cov, 
                                        num_samples)

mean_est = np.mean(samples, axis=0)
cov_est = np.cov(samples, rowvar=False)



In [6]:
print("Mean estimate: ", mean_est)
print("Covariance estimate: ", cov_est)
print("Covariance difference: ", cov_diff)


Mean estimate:  [-0.00201796 -0.02486287 -0.01088437]
Covariance estimate:  [[0.95622445 0.09198735 0.11412299]
 [0.09198735 0.95401337 0.06350824]
 [0.11412299 0.06350824 1.0571634 ]]
Covariance difference:  [[ 0.27066007  0.18482682  0.10788968]
 [ 0.18482682 -0.14532262 -0.25984144]
 [ 0.10788968 -0.25984144 -0.50041999]]


In [7]:
# now we remove columns
selected_samples = samples[:, :2]

# re estimate the mean and covariance
mean_est_selected = np.mean(selected_samples, axis=0)
cov_est_selected = np.cov(selected_samples, rowvar=False)

print("Mean estimate: ", mean_est_selected)
print("Covariance estimate: ", cov_est_selected)

Mean estimate:  [-0.00201796 -0.02486287]
Covariance estimate:  [[0.95622445 0.09198735]
 [0.09198735 0.95401337]]


what did I do? 
I created a multivariate distributions; sample some data points and calculate their covariance matrices across the distributions. 

this suggests that with a subset of features, you can estimate the submatrix out of the full matrix. Is this what we are doing? 

but we actually want to observe is how to estimate the measurement matrices

## set up the observation matrix

In [247]:
C = np.array([[1, 1, 1], 
              [2, 2, 2],
              [3, 3, 3]
              ])

Q = np.array([[1, 0.1, 0.5], 
              [0.1, 1, 0.5],
              [0.5, 0.5, 10]
            ])

y = C @ samples.T + np.random.multivariate_normal([0, 0, 0], 
                                                  Q, 
                                                  num_samples).T
print("samples: ", samples.shape, )
print("y: ", y.shape)

C_est = np.linalg.lstsq(samples, 
                        y.T, 
                        rcond=None)[0].T

Q_est = (y - C_est @ samples.T) \
        @ (y - C_est @ samples.T).T / num_samples


print("C: ", C)
print("C_est: ", C_est)

print("Q: ", Q)
print("Q_est: ", Q_est)

samples:  (1000, 3)
y:  (3, 1000)
C:  [[1 1 1]
 [2 2 2]
 [3 3 3]]
C_est:  [[1.02763249 1.075206   1.08803623]
 [2.00548362 2.00305926 1.9404042 ]
 [3.01378881 2.94580197 2.95554609]]
Q:  [[ 1.   0.1  0.5]
 [ 0.1  1.   0.5]
 [ 0.5  0.5 10. ]]
Q_est:  [[1.03329984 0.12240095 0.45200257]
 [0.12240095 0.9792542  0.53357649]
 [0.45200257 0.53357649 9.32942494]]


here, we show that we can estimate the ground truth C and Q

In [248]:
# how would it change if we remove the last column
y_selected = y[:2, :]

selected_samples = samples

C_est_selected = np.linalg.lstsq(selected_samples,
                                    y_selected.T, 
                                    rcond=None)[0].T
print("C_est_selected: ", C_est_selected)

Q_est_selected = (y_selected - C_est_selected @ selected_samples.T) \
                    @ (y_selected - C_est_selected @ selected_samples.T).T / num_samples

print("C_est_selected: ", C_est_selected)
print("Q_est_selected: ", Q_est_selected)
print("Q_est - Q_est_selected: ", np.linalg.norm(Q_est) - np.linalg.norm(Q_est_selected))
print()
print("|Q_sub - Q_est|: ", np.linalg.norm(Q_est[:2, :][:,:2]) - np.linalg.norm(Q_est_selected))


C_est_selected:  [[1.02763249 1.075206   1.08803623]
 [2.00548362 2.00305926 1.9404042 ]]
C_est_selected:  [[1.02763249 1.075206   1.08803623]
 [2.00548362 2.00305926 1.9404042 ]]
Q_est_selected:  [[1.03329984 0.12240095]
 [0.12240095 0.9792542 ]]
Q_est - Q_est_selected:  8.056580148817178

|Q_sub - Q_est|:  0.0


so deleting a feature does not change the estimation of the observation matrix  

